# Data audit & cleaning
This notebook performs the data audit and initial cleaning steps for the raw dataset located in `../data/raw.csv`.


In [3]:
# Imports and display settings
import pandas as pd
from pathlib import Path
pd.set_option('display.float_format', lambda v: f'{v:,.2f}')

data_path = Path('..') / 'data' / 'raw.csv'

# Load the raw dataset
df_raw = pd.read_csv(data_path)
print('Loaded data shape:', df_raw.shape)
df_raw.head()


Loaded data shape: (96, 19)


,Date,ffpi_food,ffpi_cereals,ffpi_veg_oils,ffpi_dairy,ffpi_meat,ffpi_sugar,bdi_price,gat_land_ocean,gat_land,gat_ocean,ffpi_Energy_Consumption,Engergy Imported,ffpi_USD/HKD_Rate,USD/HKD Rate,ipi_food,rs_Dairy_Products,rs_Fresh,wpm_fish
0,01/01/2018,96.70,95.20,98.25,106.00,95.30,87.20,"1,152.00",0.84,1.44,0.58,10 954,29 787,7.82,7.80,86.80,"1,957.00",961.00,36.30
1,02/01/2018,97.80,98.40,95.60,108.80,96.60,83.90,"1,192.00",0.89,1.62,0.57,9 787,42 998,7.83,7.80,87.10,"2,058.00",931.00,38.21
2,03/01/2018,98.80,101.80,95.36,111.30,96.60,80.90,"1,055.00",0.88,1.49,0.60,11 151,29 108,7.85,7.80,86.80,"1,867.00",925.00,35.25
3,04/01/2018,98.40,104.00,94.02,110.00,95.40,76.80,"1,341.00",0.91,1.46,0.66,11 593,23 425,7.85,7.80,86.80,"1,781.00",933.00,33.06
4,05/01/2018,98.50,105.50,92.15,111.40,94.90,76.40,"1,090.00",0.84,1.24,0.66,15 627,37 351,7.85,7.80,86.70,"1,992.00","1,006.00",35.00


In [4]:
# Basic structure and completeness checks
print('Data types:\n', df_raw.dtypes, '\n')
print('Row/column counts:', df_raw.shape)
print('Missing values per column:\n', df_raw.isna().sum(), '\n')
print('Numeric summary statistics:\n', df_raw.describe(include='all'))

Data types:
 Date                         object
ffpi_food                   float64
ffpi_cereals                float64
ffpi_veg_oils               float64
ffpi_dairy                  float64
ffpi_meat                   float64
ffpi_sugar                  float64
bdi_price                    object
gat_land_ocean              float64
gat_land                    float64
gat_ocean                   float64
ffpi_Energy_Consumption      object
Engergy Imported             object
ffpi_USD/HKD_Rate           float64
USD/HKD Rate                float64
ipi_food                    float64
rs_Dairy_Products           float64
rs_Fresh                    float64
wpm_fish                    float64
dtype: object 

Row/column counts: (96, 19)
Missing values per column:
 Date                        1
ffpi_food                   2
ffpi_cereals                2
ffpi_veg_oils               2
ffpi_dairy                  2
ffpi_meat                   2
ffpi_sugar                  2
bdi_price            

In [5]:
# Duplicate check
duplicate_rows = df_raw.duplicated()
print('Number of duplicate rows:', duplicate_rows.sum())
if duplicate_rows.any():
    display(df_raw[duplicate_rows].head())


Number of duplicate rows: 0


In [6]:
# Quick look at string-based columns to understand formatting quirks
string_cols = [c for c in df_raw.columns if df_raw[c].dtype == 'object']
for col in string_cols:
    print(f'\nSample values for {col}:')
    display(df_raw[col].dropna().head())


Sample values for Date:


0    01/01/2018
1    02/01/2018
2    03/01/2018
3    04/01/2018
4    05/01/2018
Name: Date, dtype: object


Sample values for bdi_price:


0    1,152.00
1    1,192.00
2    1,055.00
3    1,341.00
4    1,090.00
Name: bdi_price, dtype: object


Sample values for ffpi_Energy_Consumption :


0    10 954
1     9 787
2    11 151
3    11 593
4    15 627
Name: ffpi_Energy_Consumption , dtype: object


Sample values for Engergy Imported :


0    29 787
1    42 998
2    29 108
3    23 425
4    37 351
Name: Engergy Imported , dtype: object

In [7]:

# Cleaning steps
df_clean = df_raw.copy()

# 1) Standardize column names (strip spaces, fix typos, snake_case for consistency)
df_clean.columns = [c.strip() for c in df_clean.columns]
rename_map = {
    'Date': 'date',
    'ffpi_Energy_Consumption': 'ffpi_energy_consumption',
    'Engergy Imported': 'energy_imported',
    'ffpi_USD/HKD_Rate': 'ffpi_usd_hkd_rate',
    'USD/HKD Rate': 'usd_hkd_rate',
    'rs_Dairy_Products': 'rs_dairy_products',
    'rs_Fresh': 'rs_fresh',
}
df_clean = df_clean.rename(columns=rename_map)

# 2) Trim whitespace inside string fields to remove inconsistent labels
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = (
            df_clean[col]
            .astype(str)
            .str.strip()
            .replace({'': pd.NA, 'nan': pd.NA})
        )

# 3) Convert date column to datetime (month/day/year observed), normalize to month start, drop unparsed rows
if 'date' in df_clean.columns:
    df_clean['date'] = pd.to_datetime(df_clean['date'], format='%m/%d/%Y', errors='coerce')
    df_clean['date'] = df_clean['date'].dt.to_period('M').dt.to_timestamp()
    df_clean = df_clean[df_clean['date'].notna()]

# 4) Convert numeric-looking object columns by removing commas/spaces
for col in ['bdi_price', 'ffpi_energy_consumption', 'energy_imported', 'ffpi_usd_hkd_rate']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(
            df_clean[col].astype(str).str.replace(r'[\s,]+', '', regex=True),
            errors='coerce'
        )

# 5) Drop fully empty rows and remove columns that are entirely null or constant
df_clean = df_clean.dropna(how='all')
all_null_cols = [c for c in df_clean.columns if df_clean[c].isna().all()]
if all_null_cols:
    df_clean = df_clean.drop(columns=all_null_cols)

constant_cols = [c for c in df_clean.columns if df_clean[c].nunique(dropna=False) == 1]
if constant_cols:
    df_clean = df_clean.drop(columns=constant_cols)

# 6) Flag potential outliers using IQR (flag only; do not drop)
numeric_cols = df_clean.select_dtypes(include='number').columns
iqr_flags = pd.Series(False, index=df_clean.index)
for col in numeric_cols:
    q1 = df_clean[col].quantile(0.25)
    q3 = df_clean[col].quantile(0.75)
    iqr = q3 - q1
    if pd.isna(iqr):
        continue
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    iqr_flags |= (df_clean[col] < lower) | (df_clean[col] > upper)
df_clean['flag_iqr_outlier'] = iqr_flags

# 7) Flag rows with any negative numeric values (indices typically non-negative; keeping rows for review)
negative_flags = (df_clean[numeric_cols] < 0).any(axis=1)
df_clean['flag_negative_values'] = negative_flags

df_clean.head()


,date,ffpi_food,ffpi_cereals,ffpi_veg_oils,ffpi_dairy,ffpi_meat,ffpi_sugar,bdi_price,gat_land_ocean,gat_land,...,ffpi_energy_consumption,energy_imported,ffpi_usd_hkd_rate,usd_hkd_rate,ipi_food,rs_dairy_products,rs_fresh,wpm_fish,flag_iqr_outlier,flag_negative_values
0,2018-01-01,96.70,95.20,98.25,106.00,95.30,87.20,"1,152.00",0.84,1.44,...,NaN,NaN,7.82,7.80,86.80,"1,957.00",961.00,36.30,False,False
1,2018-02-01,97.80,98.40,95.60,108.80,96.60,83.90,"1,192.00",0.89,1.62,...,NaN,NaN,7.83,7.80,87.10,"2,058.00",931.00,38.21,False,False
2,2018-03-01,98.80,101.80,95.36,111.30,96.60,80.90,"1,055.00",0.88,1.49,...,NaN,NaN,7.85,7.80,86.80,"1,867.00",925.00,35.25,False,False
3,2018-04-01,98.40,104.00,94.02,110.00,95.40,76.80,"1,341.00",0.91,1.46,...,NaN,NaN,7.85,7.80,86.80,"1,781.00",933.00,33.06,False,False
4,2018-05-01,98.50,105.50,92.15,111.40,94.90,76.40,"1,090.00",0.84,1.24,...,NaN,NaN,7.85,7.80,86.70,"1,992.00","1,006.00",35.00,False,False


In [8]:
# Post-cleaning sanity checks
print('Cleaned data types:\n', df_clean.dtypes, '\n')
print('Missing values after cleaning:\n', df_clean.isna().sum(), '\n')
print('Duplicate rows after cleaning:', df_clean.duplicated().sum())

# Category level counts (using the date period to understand coverage)
if 'date' in df_clean.columns:
    monthly_counts = df_clean['date'].dt.to_period('M').value_counts().sort_index()
    print('\nRecords per month (after parsing dates):')
    display(monthly_counts)

# Surface any rows flagged for outliers or negatives without dropping them
flagged = df_clean[df_clean[['flag_iqr_outlier', 'flag_negative_values']].any(axis=1)]
print('\nRows flagged for potential review (kept in dataset):', len(flagged))
display(flagged.head())

Cleaned data types:
 date                       datetime64[ns]
ffpi_food                         float64
ffpi_cereals                      float64
ffpi_veg_oils                     float64
ffpi_dairy                        float64
ffpi_meat                         float64
ffpi_sugar                        float64
bdi_price                         float64
gat_land_ocean                    float64
gat_land                          float64
gat_ocean                         float64
ffpi_energy_consumption           float64
energy_imported                   float64
ffpi_usd_hkd_rate                 float64
usd_hkd_rate                      float64
ipi_food                          float64
rs_dairy_products                 float64
rs_fresh                          float64
wpm_fish                          float64
flag_iqr_outlier                     bool
flag_negative_values                 bool
dtype: object 

Missing values after cleaning:
 date                        1
ffpi_food          

date
2018-01    1
2018-02    1
2018-03    1
2018-04    1
2018-05    1
          ..
2025-07    1
2025-08    1
2025-09    1
2025-10    1
2025-11    1
Freq: M, Name: count, Length: 95, dtype: int64


Rows flagged for potential review (kept in dataset): 12


,date,ffpi_food,ffpi_cereals,ffpi_veg_oils,ffpi_dairy,ffpi_meat,ffpi_sugar,bdi_price,gat_land_ocean,gat_land,...,ffpi_energy_consumption,energy_imported,ffpi_usd_hkd_rate,usd_hkd_rate,ipi_food,rs_dairy_products,rs_fresh,wpm_fish,flag_iqr_outlier,flag_negative_values
12,2019-01-01,93.10,101.80,80.26,100.90,91.60,79.30,668.00,0.93,1.47,...,NaN,NaN,7.85,7.80,87.70,"2,226.00",957.00,36.93,True,False
24,2020-01-01,102.50,100.70,108.73,103.80,103.50,87.50,487.00,1.16,1.98,...,NaN,NaN,7.77,7.80,88.80,"2,439.00","1,096.00",34.62,True,False
26,2020-03-01,95.20,98.00,85.42,101.60,99.50,73.90,626.00,1.14,1.90,...,NaN,NaN,7.75,7.80,88.90,"2,091.00","1,416.00",33.74,True,False
30,2020-07-01,93.80,97.30,93.20,101.80,91.40,76.00,"1,350.00",0.92,1.26,...,NaN,NaN,7.75,7.80,88.90,"2,390.00","1,368.00",32.50,True,False
31,2020-08-01,95.70,99.20,98.70,102.10,91.40,81.10,"1,488.00",0.91,1.25,...,NaN,NaN,7.75,7.80,89.40,"2,058.00","1,389.00",36.22,True,False


In [9]:

# Optional: save cleaned data for downstream steps (CSV + Parquet)
import sys, subprocess

clean_dir = Path('..') / 'data' / 'clean'
clean_dir.mkdir(parents=True, exist_ok=True)

csv_path = clean_dir / 'data_clean.csv'
parquet_path = clean_dir / 'data_clean.parquet'
legacy_csv = Path('..') / 'data' / 'cleaned.csv'

df_clean.to_csv(csv_path, index=False)
df_clean.to_csv(legacy_csv, index=False)

try:
    import pyarrow  # noqa: F401
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyarrow'])

df_clean.to_parquet(parquet_path, index=False)

print('Cleaned dataset saved to', csv_path)
print('Parquet dataset saved to', parquet_path)
print('Legacy CSV saved to', legacy_csv)


Cleaned dataset saved to ..\data\cleaned.csv
